In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = '/kaggle/input/titanic-dataset-from-kaggle/train.csv'

data = pd.read_csv(data) 
data.head() 

# Drop the columns that are unnecessary for Machine learning process

In [ ]:
df = data.drop(['Name', 'Ticket','Cabin'],axis=1) 


In [ ]:
#No. of columns and rows
df.shape

In [ ]:
#Reduce null values
print(df.isnull().sum()) 
df.dropna(inplace = True) 

In [ ]:
print(df.columns)

# Since Sexal and embarked columns is filled with object data type, so we need to convert it into numerical form

In [ ]:
df.head()
sex = pd.get_dummies(df.Sex) 

sex.drop('female', inplace = True) 

In [ ]:
df['Sex'] = sex
df.head()

In [ ]:
embarked=pd.get_dummies(df.Embarked)
embarked.head() 

embarked.drop('S',axis=1,inplace=True) 

In [ ]:
#Adding embarked dataFrame with df
embarked.head()

df = pd.concat([df, embarked],axis=1) 

In [ ]:
df.head()
df.drop('Embarked', axis=1, inplace=True) 

In [ ]:
print(df.head()) 

X = df.drop('Survived', axis=1) 
y = df.Survived



In [ ]:
#feature data.
X.head() 



# Machine Learning process

1. split data into train 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

lg_model = LogisticRegression(max_iter=40) 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1) 

lg_model.fit(X_train,y_train) 

prediction = lg_model.predict(X_test) 

In [ ]:
#evaluating the accuracy of the model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, prediction) 

# Check how much the predicted data and actual data are matched

In [ ]:
prediction


In [ ]:
y_test.head(10) 


# Cross Validation score

# Using different methods to increase the accuracy of the model

In [ ]:
from sklearn.model_selection import cross_val_score
avg_score = cross_val_score(LogisticRegression(solver='liblinear', 
                                   multi_class='auto', 
                                   max_iter=45), X_train, y_train, cv=5) 
avg_score.mean() 


# define the function for avoiding the repitation while coding

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train) 
    return model.score(X_test, y_test) 



In [ ]:
score_lg = get_score(LogisticRegression(solver='liblinear',max_iter=45, multi_class='auto'), 
    X_train, X_test, y_train, 
    y_test)
score_rg = get_score(RandomForestRegressor(), 
            X_train,X_test,y_train,y_test)         
score_svm = get_score(SVC(kernel='rbf'), X_train, X_test, 
                      y_train, y_test) 


In [ ]:
print(score_lg) 
print(score_rg) 
print(score_svm) 


In [ ]:
lg_score = cross_val_score(LogisticRegression( solver = 'liblinear',max_iter=45, multi_class='auto'),X_train, y_train, cv=5) 
Rfr_score = cross_val_score(RandomForestRegressor(max_leaf_nodes=100),X_train, y_train, cv=5) 
svm_score = cross_val_score(SVC(),X_train, y_train, cv=5) 

In [ ]:
lg_score


In [ ]:
#creating the dataFrame for the result of cross_value score
my_data = pd.DataFrame({'Lg_score':lg_score,'Rfr_score': Rfr_score,'SVM_score':svm_score}) 

In [ ]:
my_data


In [ ]:
my_data.to_csv('submission.csv',index=False)

# Define function to calculate cross val score and try to evaluate the result using hyper parameter tuning concept


In [ ]:
from sklearn.model_selection import cross_val_score
def score_value(n_estimators):
    
    result = RandomForestRegressor(n_estimators, random_state =0) 
    score = -1* cross_val_score(result, X_train, y_train, cv=5, scoring='neg_mean_absolute_error') 
    return score.mean() 

# Here, we are calculating the mean absolute error of the model rather than score

In [ ]:
print(score_value(50)) 
#using dictionary to fit the value
results={}
for i in range(50, 401,50):
    results[i]=score_value(i) 
   # print(results) 
    
print(results) 

In [ ]:
for key, value in results.items():
    print(key, value) 

# XGboost - Xtreme Gradient Boosting, For optimizing MAE(Mean Absolute Error) at minimum Point



In [ ]:
from xgboost import XGBRegressor

my_model = XGBRegressor(n_estimators=1000,learning_rate= 0.001) 
my_model.fit(X_train, y_train) 

In [ ]:
from sklearn.metrics import mean_absolute_error

preds = my_model.predict(X_test) 
MAE = mean_absolute_error(y_test, preds) 


In [ ]:
MAE


In [ ]:
score = my_model.score(X_test, y_test) 
score


### In this case getting proper results using GBosst regressor seems to be difficult than approaches used above